In [4]:
# Importing Libraries
from collections import defaultdict
from pathlib import Path
import numpy as np
import pandas as pd
from collections import namedtuple
from tabulate import tabulate

import torch
import os

from adversarial_debiasing import AdversarialDebiasing
from load_data import load_data, transform_data, Datapoint

from load_vectors import load_pretrained_vectors, load_vectors
import config
import utility_functions

import gensim
import gzip

In [5]:
# For autoreloading changes made in other python scripts
%load_ext autoreload
%autoreload 2

In [6]:
WORD2VEC_FILE = "data/GoogleNews-vectors-negative300.bin.gz"

In [7]:
def load_word2vec_format(f, max_num_words=None):
          """Loads word2vec data from a file handle.

          Similar to gensim.models.keyedvectors.KeyedVectors.load_word2vec_format
          but takes a file handle as input rather than a filename. This lets us use
          GFile. Also only accepts binary files.

          Args:
            f: file handle
            max_num_words: number of words to load. If None, load all.

          Returns:
            Word2vec data as keyedvectors.EuclideanKeyedVectors.
          """
          header = f.readline()
          vocab_size, vector_size = (
              int(x) for x in header.rstrip().split())  # throws for invalid file format
          print("vector_size",vector_size)
          result = gensim.models.keyedvectors.EuclideanKeyedVectors()
          num_words = 0
          result.vector_size = vector_size
          result.syn0 = np.zeros((vocab_size, vector_size), dtype=np.float32)

          def add_word(word, weights):
            word_id = len(result.vocab)
            if word in result.vocab:
              print("duplicate word '%s', ignoring all but first", word)
              return
            result.vocab[word] = gensim.models.keyedvectors.Vocab(
                index=word_id, count=vocab_size - word_id)
            result.syn0[word_id] = weights
            result.index2word.append(word)

          if max_num_words and max_num_words < vocab_size:
            num_embeddings = max_num_words
          else:
            num_embeddings = vocab_size
          print("Loading ",num_embeddings," embeddings")

          binary_len = np.dtype(np.float32).itemsize * vector_size
          for _ in range(vocab_size):
            # mixed text and binary: read text first, then binary
            word = []
            while True:
              ch = f.read(1)
              if ch == b' ':
                break
              if ch == b'':
                raise EOFError("unexpected end of input; is count incorrect or file otherwise damaged?")
              if ch != b'\n':  # ignore newlines in front of words (some binary files have)
                word.append(ch)
            word = gensim.utils.to_unicode(b''.join(word), encoding='utf-8', errors='strict')
            weights = np.frombuffer(f.read(binary_len), dtype=np.float32)
            add_word(word, weights)
            num_words = num_words + 1
            if max_num_words and num_words == max_num_words:
              break
          if result.syn0.shape[0] != len(result.vocab):
            print(
                "duplicate words detected, shrinking matrix size from %i to %i",
                result.syn0.shape[0], len(result.vocab))
          result.syn0 = np.ascontiguousarray(result.syn0[:len(result.vocab)])
          assert (len(result.vocab), vector_size) == result.syn0.shape

          print("loaded %s matrix", result.syn0.shape)
          return result

In [8]:
with gzip.GzipFile(fileobj=open(WORD2VEC_FILE, "rb", buffering=0)) as f:
    word_vectors = load_word2vec_format(f, max_num_words=2000000)

vector_size 300
Loading  2000000  embeddings
duplicate words detected, shrinking matrix size from %i to %i 3000000 2000000
loaded %s matrix (2000000, 300)


In [9]:
# Loading the word vectors dictionary
word_vectors = load_pretrained_vectors(config.wiki_embedding_data_path, config.save_dir, config.wiki_save_file, config.use_glove)

FileNotFoundError: [Errno 2] No such file or directory: 'data/enwiki_20180420_100d.txt'

In [9]:
# Load the google analogies training dataset:
analogy_dataset = load_data()
analogy_dataset

[Raw_Datapoint(x1='Athens', x2='Greece', x3='Baghdad', y='Iraq', task='capital-common-countries'),
 Raw_Datapoint(x1='Athens', x2='Greece', x3='Bangkok', y='Thailand', task='capital-common-countries'),
 Raw_Datapoint(x1='Athens', x2='Greece', x3='Beijing', y='China', task='capital-common-countries'),
 Raw_Datapoint(x1='Athens', x2='Greece', x3='Berlin', y='Germany', task='capital-common-countries'),
 Raw_Datapoint(x1='Athens', x2='Greece', x3='Bern', y='Switzerland', task='capital-common-countries'),
 Raw_Datapoint(x1='Athens', x2='Greece', x3='Cairo', y='Egypt', task='capital-common-countries'),
 Raw_Datapoint(x1='Athens', x2='Greece', x3='Canberra', y='Australia', task='capital-common-countries'),
 Raw_Datapoint(x1='Athens', x2='Greece', x3='Hanoi', y='Vietnam', task='capital-common-countries'),
 Raw_Datapoint(x1='Athens', x2='Greece', x3='Havana', y='Cuba', task='capital-common-countries'),
 Raw_Datapoint(x1='Athens', x2='Greece', x3='Helsinki', y='Finland', task='capital-common-cou

In [10]:
# Transform the data such that it includes the embeddings of the words in consideration
transformed_analogy_dataset, gender_subspace = transform_data(word_vectors, analogy_dataset, use_boluk = False)
word_embedding_dim = transformed_analogy_dataset[0].gt_embedding.shape[0]

In [11]:
# Testing the transformed analogy dataset
print(transformed_analogy_dataset[0].analogy_embeddings.shape)
print(transformed_analogy_dataset[0].gt_embedding.shape)
print(transformed_analogy_dataset[0].protected.shape)

(900,)
(300,)
(1,)


In [13]:
word_embedding_dim = transformed_analogy_dataset[0].gt_embedding.shape[0]
# Training the variant of the model without debiasing
non_debiased_model = AdversarialDebiasing(
    word_embedding_dim=word_embedding_dim,
    debias=False,
    num_epochs=50,
    gender_subspace=gender_subspace
)
non_debiased_model.fit(dataset=transformed_analogy_dataset)

[0/50] Running epoch


NameError: name 'quit' is not defined

In [20]:
W1 = non_debiased_model.get_model_weights()
print(np.dot(W1.detach().numpy().T,gender_subspace.T))

[[0.09051673]]


In [41]:
gender_subspace.shape

(1, 300)

In [67]:
# Training the variant of the model with debiasing
# debiased_model = AdversarialDebiasing(
#     word_embedding_dim=word_embedding_dim,
#     num_epochs=500,
#     debias=True,
#     gender_subspace=gender_subspace,
#     batch_size=256,
#     adversary_loss_weight=0.1,
#     classifier_learning_rate = 2 ** -8,
#     adversary_learning_rate = 2 ** -8
# )
debiased_model = AdversarialDebiasing(
    word_embedding_dim=word_embedding_dim,
    num_epochs=500,
    debias=True,
    gender_subspace=gender_subspace,
    batch_size=256,
    adversary_loss_weight=0.1,
    classifier_learning_rate = 2 ** -6,
    adversary_learning_rate = 2 ** -6
)

debiased_model.fit(dataset=transformed_analogy_dataset)

[0/500] Running epoch
epoch 0; iter: 0; batch classifier loss: 0.040327; batch adversarial loss: 0.030493
epoch 0; iter: 10; batch classifier loss: 0.039214; batch adversarial loss: 0.034037
epoch 0; iter: 20; batch classifier loss: 0.036291; batch adversarial loss: 0.075652
epoch 0; iter: 30; batch classifier loss: 0.148507; batch adversarial loss: 2.361297
epoch 0; iter: 40; batch classifier loss: 0.535863; batch adversarial loss: 0.888197
epoch 0; iter: 50; batch classifier loss: 0.183061; batch adversarial loss: 0.332109
||w||: 2.174010753631592
||w2||: 2.007164239883423
w.T g: [[0.27487676]]
[1/500] Running epoch
epoch 1; iter: 0; batch classifier loss: 0.109137; batch adversarial loss: 0.395982
epoch 1; iter: 10; batch classifier loss: 0.059609; batch adversarial loss: 2.302217
epoch 1; iter: 20; batch classifier loss: 0.461144; batch adversarial loss: 0.860398
epoch 1; iter: 30; batch classifier loss: 1.724663; batch adversarial loss: 0.478470
epoch 1; iter: 40; batch classifier

epoch 15; iter: 30; batch classifier loss: 0.037273; batch adversarial loss: 0.053396
epoch 15; iter: 40; batch classifier loss: 0.037678; batch adversarial loss: 0.059625
epoch 15; iter: 50; batch classifier loss: 0.037276; batch adversarial loss: 0.055768
[16/500] Running epoch
epoch 16; iter: 0; batch classifier loss: 0.039109; batch adversarial loss: 0.046397
epoch 16; iter: 10; batch classifier loss: 0.040425; batch adversarial loss: 0.050679
epoch 16; iter: 20; batch classifier loss: 0.037717; batch adversarial loss: 0.051692
epoch 16; iter: 30; batch classifier loss: 0.037383; batch adversarial loss: 0.067998
epoch 16; iter: 40; batch classifier loss: 0.037204; batch adversarial loss: 0.051346
epoch 16; iter: 50; batch classifier loss: 0.038242; batch adversarial loss: 0.053789
[17/500] Running epoch
epoch 17; iter: 0; batch classifier loss: 0.037816; batch adversarial loss: 0.047932
epoch 17; iter: 10; batch classifier loss: 0.037960; batch adversarial loss: 0.057060
epoch 17; 

epoch 31; iter: 10; batch classifier loss: 0.038582; batch adversarial loss: 0.037158
epoch 31; iter: 20; batch classifier loss: 0.037240; batch adversarial loss: 0.038853
epoch 31; iter: 30; batch classifier loss: 0.041650; batch adversarial loss: 0.036140
epoch 31; iter: 40; batch classifier loss: 0.039817; batch adversarial loss: 0.031727
epoch 31; iter: 50; batch classifier loss: 0.037454; batch adversarial loss: 0.028943
[32/500] Running epoch
epoch 32; iter: 0; batch classifier loss: 0.037247; batch adversarial loss: 0.028948
epoch 32; iter: 10; batch classifier loss: 0.037336; batch adversarial loss: 0.038201
epoch 32; iter: 20; batch classifier loss: 0.037989; batch adversarial loss: 0.029963
epoch 32; iter: 30; batch classifier loss: 0.037868; batch adversarial loss: 0.028966
epoch 32; iter: 40; batch classifier loss: 0.037890; batch adversarial loss: 0.033809
epoch 32; iter: 50; batch classifier loss: 0.037136; batch adversarial loss: 0.026785
[33/500] Running epoch
epoch 33;

epoch 46; iter: 30; batch classifier loss: 0.036477; batch adversarial loss: 0.024078
epoch 46; iter: 40; batch classifier loss: 0.036915; batch adversarial loss: 0.024886
epoch 46; iter: 50; batch classifier loss: 0.036371; batch adversarial loss: 0.024623
[47/500] Running epoch
epoch 47; iter: 0; batch classifier loss: 0.038069; batch adversarial loss: 0.022509
epoch 47; iter: 10; batch classifier loss: 0.037159; batch adversarial loss: 0.021662
epoch 47; iter: 20; batch classifier loss: 0.036852; batch adversarial loss: 0.021403
epoch 47; iter: 30; batch classifier loss: 0.036001; batch adversarial loss: 0.026396
epoch 47; iter: 40; batch classifier loss: 0.037143; batch adversarial loss: 0.024533
epoch 47; iter: 50; batch classifier loss: 0.038081; batch adversarial loss: 0.021424
[48/500] Running epoch
epoch 48; iter: 0; batch classifier loss: 0.038379; batch adversarial loss: 0.023278
epoch 48; iter: 10; batch classifier loss: 0.038973; batch adversarial loss: 0.023350
epoch 48; 

epoch 61; iter: 30; batch classifier loss: 0.039215; batch adversarial loss: 0.023563
epoch 61; iter: 40; batch classifier loss: 0.036207; batch adversarial loss: 0.021414
epoch 61; iter: 50; batch classifier loss: 0.036493; batch adversarial loss: 0.021226
[62/500] Running epoch
epoch 62; iter: 0; batch classifier loss: 0.037449; batch adversarial loss: 0.020198
epoch 62; iter: 10; batch classifier loss: 0.036582; batch adversarial loss: 0.021288
epoch 62; iter: 20; batch classifier loss: 0.036178; batch adversarial loss: 0.019353
epoch 62; iter: 30; batch classifier loss: 0.038888; batch adversarial loss: 0.025021
epoch 62; iter: 40; batch classifier loss: 0.037660; batch adversarial loss: 0.021394
epoch 62; iter: 50; batch classifier loss: 0.040290; batch adversarial loss: 0.020765
[63/500] Running epoch
epoch 63; iter: 0; batch classifier loss: 0.037768; batch adversarial loss: 0.023223
epoch 63; iter: 10; batch classifier loss: 0.034802; batch adversarial loss: 0.020304
epoch 63; 

...

epoch 425; iter: 20; batch classifier loss: 0.038568; batch adversarial loss: 0.026110
epoch 425; iter: 30; batch classifier loss: 0.037874; batch adversarial loss: 0.022140
epoch 425; iter: 40; batch classifier loss: 0.038017; batch adversarial loss: 0.026420
epoch 425; iter: 50; batch classifier loss: 0.037659; batch adversarial loss: 0.024347
[426/500] Running epoch
epoch 426; iter: 0; batch classifier loss: 0.036841; batch adversarial loss: 0.023020
epoch 426; iter: 10; batch classifier loss: 0.036660; batch adversarial loss: 0.024381
epoch 426; iter: 20; batch classifier loss: 0.038064; batch adversarial loss: 0.025227
epoch 426; iter: 30; batch classifier loss: 0.035793; batch adversarial loss: 0.022195
epoch 426; iter: 40; batch classifier loss: 0.038122; batch adversarial loss: 0.025595
epoch 426; iter: 50; batch classifier loss: 0.036148; batch adversarial loss: 0.019879
[427/500] Running epoch
epoch 427; iter: 0; batch classifier loss: 0.037354; batch adversarial loss: 0.0240

epoch 440; iter: 40; batch classifier loss: 0.037192; batch adversarial loss: 0.021010
epoch 440; iter: 50; batch classifier loss: 0.037452; batch adversarial loss: 0.022748
||w||: 1.2978535890579224
||w2||: 0.5965328812599182
w.T g: [[0.30422238]]
[441/500] Running epoch
epoch 441; iter: 0; batch classifier loss: 0.037778; batch adversarial loss: 0.025982
epoch 441; iter: 10; batch classifier loss: 0.037931; batch adversarial loss: 0.025827
epoch 441; iter: 20; batch classifier loss: 0.037546; batch adversarial loss: 0.023812
epoch 441; iter: 30; batch classifier loss: 0.037244; batch adversarial loss: 0.022601
epoch 441; iter: 40; batch classifier loss: 0.036675; batch adversarial loss: 0.027448
epoch 441; iter: 50; batch classifier loss: 0.038624; batch adversarial loss: 0.021556
[442/500] Running epoch
epoch 442; iter: 0; batch classifier loss: 0.039040; batch adversarial loss: 0.020798
epoch 442; iter: 10; batch classifier loss: 0.036873; batch adversarial loss: 0.023568
epoch 442

epoch 455; iter: 30; batch classifier loss: 0.037232; batch adversarial loss: 0.026588
epoch 455; iter: 40; batch classifier loss: 0.036942; batch adversarial loss: 0.023293
epoch 455; iter: 50; batch classifier loss: 0.035075; batch adversarial loss: 0.024382
[456/500] Running epoch
epoch 456; iter: 0; batch classifier loss: 0.037932; batch adversarial loss: 0.020967
epoch 456; iter: 10; batch classifier loss: 0.037076; batch adversarial loss: 0.021163
epoch 456; iter: 20; batch classifier loss: 0.037351; batch adversarial loss: 0.023059
epoch 456; iter: 30; batch classifier loss: 0.037929; batch adversarial loss: 0.024133
epoch 456; iter: 40; batch classifier loss: 0.035176; batch adversarial loss: 0.016018
epoch 456; iter: 50; batch classifier loss: 0.038174; batch adversarial loss: 0.023803
[457/500] Running epoch
epoch 457; iter: 0; batch classifier loss: 0.038803; batch adversarial loss: 0.023348
epoch 457; iter: 10; batch classifier loss: 0.036867; batch adversarial loss: 0.0221

epoch 470; iter: 40; batch classifier loss: 0.037777; batch adversarial loss: 0.022654
epoch 470; iter: 50; batch classifier loss: 0.036902; batch adversarial loss: 0.024190
||w||: 1.2978535890579224
||w2||: 0.5965328812599182
w.T g: [[0.3042224]]
[471/500] Running epoch
epoch 471; iter: 0; batch classifier loss: 0.036333; batch adversarial loss: 0.022666
epoch 471; iter: 10; batch classifier loss: 0.038527; batch adversarial loss: 0.019739
epoch 471; iter: 20; batch classifier loss: 0.040839; batch adversarial loss: 0.024836
epoch 471; iter: 30; batch classifier loss: 0.039376; batch adversarial loss: 0.022677
epoch 471; iter: 40; batch classifier loss: 0.036864; batch adversarial loss: 0.019377
epoch 471; iter: 50; batch classifier loss: 0.037610; batch adversarial loss: 0.019916
[472/500] Running epoch
epoch 472; iter: 0; batch classifier loss: 0.035705; batch adversarial loss: 0.020338
epoch 472; iter: 10; batch classifier loss: 0.036495; batch adversarial loss: 0.023572
epoch 472;

epoch 485; iter: 40; batch classifier loss: 0.036388; batch adversarial loss: 0.024779
epoch 485; iter: 50; batch classifier loss: 0.037003; batch adversarial loss: 0.019941
[486/500] Running epoch
epoch 486; iter: 0; batch classifier loss: 0.036281; batch adversarial loss: 0.022235
epoch 486; iter: 10; batch classifier loss: 0.036313; batch adversarial loss: 0.019541
epoch 486; iter: 20; batch classifier loss: 0.039310; batch adversarial loss: 0.022718
epoch 486; iter: 30; batch classifier loss: 0.036877; batch adversarial loss: 0.022312
epoch 486; iter: 40; batch classifier loss: 0.039548; batch adversarial loss: 0.028095
epoch 486; iter: 50; batch classifier loss: 0.035223; batch adversarial loss: 0.023139
[487/500] Running epoch
epoch 487; iter: 0; batch classifier loss: 0.038163; batch adversarial loss: 0.023958
epoch 487; iter: 10; batch classifier loss: 0.038875; batch adversarial loss: 0.024517
epoch 487; iter: 20; batch classifier loss: 0.036340; batch adversarial loss: 0.0226

In [66]:
W1 = debiased_model.get_model_weights()
print(np.dot(W1.clone().cpu().detach().numpy().T,gender_subspace.T))

[[0.28268819]]


In [51]:
# Examples to test the models upon
datapoints, test_analogies = [], []
with open(os.path.join('data', 'sexism-traps.txt'), 'r') as f:
    # Reading each line
    for line in f.readlines():
        words = line.split()
        if words[0] == ':':
            continue
        test_analogies.append(words)
        word_embeddings = word_vectors[words]
        word_embeddings = np.reshape(word_embeddings, (1, -1))
        datapoints.append(word_embeddings)
datapoints = np.vstack(datapoints)
print(datapoints.shape)

(21, 900)


In [56]:
# Qualitative evaluation of the non-debiased model

# non_debiased_predictions = non_debiased_model.predict(datapoints)
features = torch.cat([torch.Tensor(x).unsqueeze_(0) for x in datapoints])
x1 = features[:, 0:word_embedding_dim]
x2 = features[:, word_embedding_dim:word_embedding_dim * 2]
x3 = features[:, word_embedding_dim * 2:word_embedding_dim * 3]

non_debiased_predictions = x2 + x3 - x1
non_debiased_predictions = non_debiased_predictions.cpu().detach().numpy()

In [57]:
non_debiased_most_similar_list = utility_functions.obtain_most_similar(non_debiased_predictions, word_vectors)

In [58]:
# Displaying the similarity list for the non-debiased model
non_debiased_most_similar_list_data_frames = []
for i in range(len(non_debiased_most_similar_list)):
    # print("{} : {} :: {} : ".format(test_analogies[i][0], test_analogies[i][1], test_analogies[i][2]))
    temp_data_frame = pd.DataFrame(non_debiased_most_similar_list[i][1:], columns = ['Neighbor', 'Similarity'])
    non_debiased_most_similar_list_data_frames.append(temp_data_frame)
    # print(tabulate(temp_data_frame, headers='keys', tablefmt='psql', showindex=False))

In [60]:
# Qualitative evaluation of the debiased model
debiased_predictions = debiased_model.predict(datapoints)
debiased_most_similar_list = utility_functions.obtain_most_similar(debiased_predictions, word_vectors)

In [61]:
# Displaying the similarity list for the debiased model
debiased_most_similar_list_data_frames = []
for i in range(len(debiased_most_similar_list)):
    # print("{} : {} :: {} : ".format(test_analogies[i][0], test_analogies[i][1], test_analogies[i][2]))
    temp_data_frame = pd.DataFrame(debiased_most_similar_list[i][1:], columns = ['Neighbor', 'Similarity'])
    debiased_most_similar_list_data_frames.append(temp_data_frame)
    # print(tabulate(temp_data_frame, headers='keys', tablefmt='psql', showindex=False))

In [62]:
# Combining the dataframes pertaining to both the variants of the model
iterables = [['Biased', 'Debiased'], ['Neighbour', 'Similarity']]
index = pd.MultiIndex.from_product(iterables)
overall_data_frames_list = []
for i in range(len(non_debiased_most_similar_list)):
    overall_list = []
    print("{} : {} :: {} : ".format(test_analogies[i][0], test_analogies[i][1], test_analogies[i][2]))
    for j in range(len(non_debiased_most_similar_list[i][1:])):
        temp_list = []
        temp_list.append(non_debiased_most_similar_list[i][j][0])
        temp_list.append(round(non_debiased_most_similar_list[i][j][1], 3))
        temp_list.append(debiased_most_similar_list[i][j][0])
        temp_list.append(round(debiased_most_similar_list[i][j][1], 3))
        overall_list.append(temp_list)
    temp_df = pd.DataFrame(overall_list, columns = index)
    # print(temp_df.to_string(index = False))
    print(tabulate(temp_df, headers = ['Biased\nNeighbour', 'Biased\nSimilarity', 'Debiased\nNeighbour', 'Debiased\nSimilarity'], tablefmt = 'psql', showindex = False))
    overall_data_frames_list.append(temp_df)

he : strong :: she : 
+-------------+--------------+-------------+--------------+
| Biased      |       Biased | Debiased    |     Debiased |
| Neighbour   |   Similarity | Neighbour   |   Similarity |
|-------------+--------------+-------------+--------------|
| strong      |        0.794 | strong      |        0.815 |
| robust      |        0.531 | stong       |        0.54  |
| stong       |        0.516 | robust      |        0.537 |
| solid       |        0.51  | solid       |        0.536 |
| stronger    |        0.504 | stronger    |        0.528 |
| weak        |        0.493 | strongest   |        0.494 |
| strongest   |        0.476 | weak        |        0.492 |
| Strong      |        0.468 | Strong      |        0.471 |
| she         |        0.436 | good        |        0.437 |
+-------------+--------------+-------------+--------------+
he : boss :: she : 
+---------------------------+--------------+-----------------+--------------+
| Biased                    |       Bias

In [ ]:
# Fake dataset for testing purposes

# embedding_dim = 100
# analogy_dataset = [
#     Datapoint(
#     analogy_embeddings=np.random.normal(0, 1, size=(3 * embedding_dim, 1)), 
#     gt_embedding=np.random.normal(0, 1, size=(embedding_dim, 1)),
#     protected_embedding=np.random.uniform(0, 1, size=(1))) for n in range(0, 1000)
# ]

In [ ]:
# Fake dataset for testing purposes

# embedding_dim = 100
# analogy_dataset = [
#     Datapoint(
#     analogy_embeddings=np.random.normal(0, 1, size=(3 * embedding_dim, 1)), 
#     gt_embedding=np.random.normal(0, 1, size=(embedding_dim, 1)),
#     protected_embedding=np.random.uniform(0, 1, size=(1))) for n in range(0, 1000)
# ]

In [ ]:
# Loading the word vectors dictionary
word_vectors = load_pretrained_vectors(config.wiki_embedding_data_path, config.save_dir, config.wiki_save_file, config.use_glove)

In [ ]:
# Load the google analogies training dataset:
analogy_dataset = load_data()
analogy_dataset

In [ ]:
# Transform the data such that it includes the embeddings of the words in consideration
transformed_analogy_dataset, gender_subspace = transform_data(word_vectors, analogy_dataset, use_boluk = False)

In [ ]:
# Testing the transformed analogy dataset
print(transformed_analogy_dataset[0].analogy_embeddings.shape)
print(transformed_analogy_dataset[0].gt_embedding.shape)
print(transformed_analogy_dataset[0].protected.shape)

In [ ]:
# Training the variant of the model without debiasing
non_debiased_model = AdversarialDebiasing(debias=False, num_epochs=50)
non_debiased_model.fit(dataset=transformed_analogy_dataset)

In [ ]:
W1 = non_debiased_model.get_model_weights()
print(np.dot(W1.detach().numpy().T,gender_subspace.T))

In [ ]:
# Training the variant of the model with debiasing
debiased_model = AdversarialDebiasing(num_epochs=500)
debiased_model.fit(dataset=transformed_analogy_dataset)

In [ ]:
W1 = debiased_model.get_model_weights()
print(np.dot(W1.detach().numpy().T,gender_subspace.T))

In [ ]:
# Examples to test the models upon
datapoints, test_analogies = [], []
with open(os.path.join('data', 'sexism-traps.txt'), 'r') as f:
    # Reading each line
    for line in f.readlines():
        words = line.split()
        if words[0] == ':':
            continue
        test_analogies.append(words)
        word_embeddings = word_vectors[words]
        word_embeddings = np.reshape(word_embeddings, (1, -1))
        datapoints.append(word_embeddings)
datapoints = np.vstack(datapoints)
print(datapoints.shape)

In [ ]:
# Qualitative evaluation of the non-debiased model
non_debiased_predictions = non_debiased_model.predict(datapoints)
non_debiased_most_similar_list = utility_functions.obtain_most_similar(non_debiased_predictions, word_vectors)

In [ ]:
# Displaying the similarity list for the non-debiased model
non_debiased_most_similar_list_data_frames = []
for i in range(len(non_debiased_most_similar_list)):
    # print("{} : {} :: {} : ".format(test_analogies[i][0], test_analogies[i][1], test_analogies[i][2]))
    temp_data_frame = pd.DataFrame(non_debiased_most_similar_list[i][1:], columns = ['Neighbor', 'Similarity'])
    non_debiased_most_similar_list_data_frames.append(temp_data_frame)
    # print(tabulate(temp_data_frame, headers='keys', tablefmt='psql', showindex=False))

In [ ]:
# Qualitative evaluation of the debiased model
debiased_predictions = debiased_model.predict(datapoints)
debiased_most_similar_list = utility_functions.obtain_most_similar(debiased_predictions, word_vectors)

In [ ]:
# Displaying the similarity list for the debiased model
debiased_most_similar_list_data_frames = []
for i in range(len(debiased_most_similar_list)):
    # print("{} : {} :: {} : ".format(test_analogies[i][0], test_analogies[i][1], test_analogies[i][2]))
    temp_data_frame = pd.DataFrame(debiased_most_similar_list[i][1:], columns = ['Neighbor', 'Similarity'])
    debiased_most_similar_list_data_frames.append(temp_data_frame)
    # print(tabulate(temp_data_frame, headers='keys', tablefmt='psql', showindex=False))

In [ ]:
# Combining the dataframes pertaining to both the variants of the model
iterables = [['Biased', 'Debiased'], ['Neighbour', 'Similarity']]
index = pd.MultiIndex.from_product(iterables)
overall_data_frames_list = []
for i in range(len(non_debiased_most_similar_list)):
    overall_list = []
    print("{} : {} :: {} : ".format(test_analogies[i][0], test_analogies[i][1], test_analogies[i][2]))
    for j in range(len(non_debiased_most_similar_list[i][1:])):
        temp_list = []
        temp_list.append(non_debiased_most_similar_list[i][j][0])
        temp_list.append(round(non_debiased_most_similar_list[i][j][1], 3))
        temp_list.append(debiased_most_similar_list[i][j][0])
        temp_list.append(round(debiased_most_similar_list[i][j][1], 3))
        overall_list.append(temp_list)
    temp_df = pd.DataFrame(overall_list, columns = index)
    # print(temp_df.to_string(index = False))
    print(tabulate(temp_df, headers = ['Biased\nNeighbour', 'Biased\nSimilarity', 'Debiased\nNeighbour', 'Debiased\nSimilarity'], tablefmt = 'psql', showindex = False))
    overall_data_frames_list.append(temp_df)

In [ ]:
# Fake dataset for testing purposes

# embedding_dim = 100
# analogy_dataset = [
#     Datapoint(
#     analogy_embeddings=np.random.normal(0, 1, size=(3 * embedding_dim, 1)), 
#     gt_embedding=np.random.normal(0, 1, size=(embedding_dim, 1)),
#     protected_embedding=np.random.uniform(0, 1, size=(1))) for n in range(0, 1000)
# ]